In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
dir_path="./gtfs"
routes_file=dir_path+"/routes.txt"

In [25]:
df = pd.read_csv(routes_file)
# lines = ["U1", "U2", "U3", "U4", "U6"]
lines = ["U6"]

df = df[df.route_type == 1]
df = df[df.route_short_name.isin(lines)].groupby('route_short_name').nth(1).reset_index()
route_ids = df[["route_id", "route_long_name"]]
df

,index,route_id,agency_id,route_short_name,route_long_name,route_type,route_color,route_text_color
0,66,21-U6-j24-2,4,U6,Siebenhirten - Floridsdorf,1,9D6830,FFFFFF


In [4]:
# trips_df = pd.read_csv(dir_path + "/trips.txt")
# stops_df = pd.read_csv(dir_path + "/stops.txt")
# stop_times_df = pd.read_csv(dir_path + "/stop_times.txt")

# j = 0
# for index, route in route_ids.iterrows():
#     metro_trips = trips_df[trips_df['route_id'].isin([route.route_id])]

#     begin = True
#     i = 0
#     trips = []
#     for index, trip in metro_trips.iterrows():
#         trip_stop_times = stop_times_df.loc[stop_times_df.trip_id == trip.trip_id]
#         stops = stops_df[stops_df.stop_id.isin(trip_stop_times.stop_id)]
#         stop_names = stops_df[['stop_id','stop_name']]
#         route_schedule = trip_stop_times.merge(stop_names, on='stop_id').drop(columns=['pickup_type', 'drop_off_type'])

#         for i, row in route_schedule.iterrows():
#             time = row['arrival_time']
#             if time.startswith('24:'):
#                 time = "00" + time[2:]
#             route_schedule.at[i, 'arrival_time'] = time            
#         route_schedule['arrival_time'] = pd.to_datetime(route_schedule['arrival_time'], format='%H:%M:%S').dt.time
#         route_schedule['departure_time'] = pd.to_datetime(route_schedule['departure_time'], format='%H:%M:%S').dt.time
#         # # route_schedule = route_schedule.sort_values(by='arrival_time')
#         # route_schedule = route_schedule.sort_values('arrival_time')
#         trips.append(route_schedule)
        
#         # display(route_schedule.iloc[[0]])
#         i += 1
#         if i == 8:
#             break
#     sorteddflist= sorted(trips, key=lambda x:x["arrival_time"].min(axis=0))
#     # print(sorteddflist)

#     j += 1
#     if j == 2:
#         break

In [26]:
trips_df = pd.read_csv(dir_path + "/trips.txt")
stops_df = pd.read_csv(dir_path + "/stops.txt")
stop_times_df = pd.read_csv(dir_path + "/stop_times.txt")

file = open('schedule_U6.json', 'w+', encoding='UTF8')
file.write("{\n")
file.write("\t\"lines\":\n")
file.write("\t[\n")

j = 0
begin_1 = True
for index, route in route_ids.iterrows():
    
    metro_trips = trips_df[trips_df['route_id'].isin([route.route_id])]
    if begin_1:
        file.write("{\n")
        begin_1 = False
    else:
        file.write(",\n{\n")
    file.write(f"\"line\": \"{route.route_long_name}\",\n")
    file.write("\"trips\":\n")
    file.write("[\n")

    begin = True
    i = 0
    trips = []
    for index, trip in metro_trips.iterrows():
        
        trip_stop_times = stop_times_df.loc[stop_times_df.trip_id == trip.trip_id]
        stops = stops_df[stops_df.stop_id.isin(trip_stop_times.stop_id)]
        stop_names = stops_df[['stop_id','stop_name']]
        route_schedule = trip_stop_times.merge(stop_names, on='stop_id').drop(columns=['pickup_type', 'drop_off_type'])

        route_schedule = route_schedule[route_schedule['arrival_time'].apply(lambda x: int(x[:2])) < 24]
        route_schedule = route_schedule[route_schedule['departure_time'].apply(lambda x: int(x[:2])) < 24]

        # route_schedule['arrival_time'] = pd.to_datetime(route_schedule['arrival_time'], format='%H:%M:%S').dt.time
        # route_schedule['departure_time'] = pd.to_datetime(route_schedule['departure_time'], format='%H:%M:%S').dt.time
        
        # route_schedule = route_schedule[route_schedule['arrival_time'].apply(lambda x: not isinstance(x, float))]
        
        trips.append(route_schedule)

        i += 1
        # if i == 300:
            # break

#     try:
#         sorteddflist = sorted(schedules[0], key=lambda x: x["arrival_time"].iloc[0])
# except Exception:
        sorteddflist = trips
        # print("catched error")
        # pass
        
        for route_schedule in sorteddflist:
            out = route_schedule.to_json(orient='records').replace(",", ",\n").replace("{", "{\n\t").replace("}", "}\n").replace("}\n,", "\n\t},").replace(",\n", ",\n\t")
            if not begin:
                out = out.replace("[", ",[\n\t")
            begin = False
            file.write(out)
        
    file.write("\n]")
    file.write("\n}")
    # j += 1
    # if j == 2:
        # break

    print(route.route_long_name)
    print(i)
        

file.write("\n]")
file.write("\n}")
file.close()

Siebenhirten - Floridsdorf
515
